In [106]:
import pandas as pd
import numpy as np

import openai
openai.api_key = 'INSERT YOUR KEY HERE'

import string

import time

In [78]:
training_data = pd.read_csv('train.csv')
testing_data = pd.read_csv('test.csv')

In [79]:
answers = pd.DataFrame(columns=['id', 'prediction'])
new_rows = []  # List to collect new rows

for index, row in testing_data.iterrows():
    response = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=[
            {
                "role": "user",
                "content": "Given this question: {question}\n\nWhich three answers are correct?\n\nA: {a} \n\nB: {b}\n\nC: {c}\n\nD: {d}\n\nE: {e}\n\n\n Return only THREE letters corresponding to your answer. For example: [B D E]. Only separate the answers with spaces. If you do not know the answer, just make a guess. \n\n\n ** Remember to answer only with the letters corresponding to your answer. **".format(
                    question=row['prompt'], 
                    a=row['A'], 
                    b=row['B'], 
                    c=row['C'], 
                    d=row['D'], 
                    e=row['E']
                )
            },
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    new_rows.append({'id': index, 'prediction': response.choices[0].message.content.translate(str.maketrans('', '', string.punctuation))})

answers = pd.concat([answers, pd.DataFrame(new_rows)], ignore_index=True)


In [81]:
answers.to_csv('submission.csv', sep=',', na_rep='NA', columns=['id', 'prediction'], header=True, encoding='utf-8', index=False)

In [85]:
answers['answer'] = training_data['answer']

In [103]:
result_rows = []
total_right = 0

for index, row in answers.iterrows():
    print('prediction: ' + row['prediction'])
    print('answer: ' + row['answer'])
    
    if row['answer'] in row['prediction']:
        total_right += 1
        result_rows.append({'result': True})
        print('CORRECT')
        
    else:
        result_rows.append({'result': False})
        print('INCORRECT')
    
    print()
    
results = pd.DataFrame(result_rows)
answers['results'] = results['result']

prediction: B D E
answer: D
CORRECT

prediction: A B E
answer: A
CORRECT

prediction: A B D
answer: A
CORRECT

prediction: A C D
answer: C
CORRECT

prediction: A D E
answer: D
CORRECT

prediction: B C E
answer: B
CORRECT

prediction: A C D
answer: A
CORRECT

prediction: B D E
answer: D
CORRECT

prediction: A B C
answer: C
CORRECT

prediction: A B E
answer: A
CORRECT

prediction: E D A
answer: E
CORRECT

prediction: A C D
answer: A
CORRECT

prediction: C E A
answer: C
CORRECT

prediction: D E A
answer: D
CORRECT

prediction: B A D
answer: B
CORRECT

prediction: B D E
answer: B
CORRECT

prediction: A C E
answer: E
CORRECT

prediction: B D E
answer: E
CORRECT

prediction: A C E
answer: A
CORRECT

prediction: E
answer: E
CORRECT

prediction: D B E
answer: D
CORRECT

prediction: B E D
answer: D
CORRECT

prediction: C D E
answer: C
CORRECT

prediction: B C E
answer: C
CORRECT

prediction: B D E
answer: E
CORRECT

prediction: E A B
answer: E
CORRECT

prediction: A C B
answer: A
CORRECT

predi

In [105]:
print('Final Results: ' + str(total_right) + '/' + str(len(answers)) + ' or ' + str(total_right/len(answers) * 100) + '%')

Final Results: 199/200 or 99.5%


GPT-4 Turbo (1106 Preview) scored **199/200** or **99.5%** on the Test Dataset

In [107]:
answers

,id,prediction,answer,results
0,0,B D E,D,True
1,1,A B E,A,True
2,2,A B D,A,True
3,3,A C D,C,True
4,4,A D E,D,True
...,...,...,...,...
195,195,C E A,C,True
196,196,B C E,B,True
197,197,A B C,B,True
198,198,A C D,D,True


In [108]:
answers[answers['results'] == False]

,id,prediction,answer,results
65,65,B C D,E,False


In [112]:
training_data[training_data['id'] == 65]

,id,prompt,A,B,C,D,E,answer
65,65,What is the relevant type of coherence for the...,Visibility,Coherence time,Spatial coherence,Coherence length,Diameter of the coherence area (Ac),E
